In [40]:
import jp_util.common_path as cp
from jp_util.pandas_util import rd_csv_sig, to_csv_sig
from jp_util.common_util import l, p
import jp_util.jap_base_class as jp
from importlib import reload
import pandas as pd

reload(cp)

<module 'jp_util.common_path' from 'D:\\02.work_project\\jp_util\\src\\jp_util\\common_path.py'>

In [39]:
# 获取13万的基础词条
df_base_word = jp.JapBaseWordClass.gen_df_base_word()
p(len(df_base_word))

133637


## 词频调整处理步骤
1. 整理2400_v3_raw.csv的词汇， 在原有的基础词条中标注，哪些是高中词汇。
2. 

In [49]:
# 1. 整理2400_v3_raw.csv的词汇， 在原有的基础词条中标注，哪些是高中词汇。
df_2400_v3 = rd_csv_sig(cp.r_word_2400_v3_csv)
p(len(df_2400_v3))

# 把2400_v3去重
df_2400_v3_drop_dup = df_2400_v3.drop_duplicates(subset=["spell"], keep="first")
p(len(df_2400_v3_drop_dup))


df_merge_base_2400_v3 = pd.merge(
    df_base_word,
    df_2400_v3_drop_dup[["spell"]],
    left_on="spell",
    right_on="spell",
    how="left",
    indicator="ck_2400_v3",
)


#标注高中
#df_merge_base_2400_v3_flag




to_csv_sig(df_merge_base_2400_v3, "d:/tmp/df_merge_base_2400_v3.csv")
l("ok")

2682
2624
----------------------------------------ok----------------------------------------


In [37]:
# 包含特殊字符的词(数字，符号，英文)
filtered_df_freq = df_base_word[
    df_base_word["spell"].str.contains(cp.filter_pattern, na=False)
].copy()
to_csv_sig(filtered_df_freq, 'd:/tmp/special_char_in_base_word.csv')
l("ok")

----------------------------------------ok----------------------------------------
